### **| Scrapping 1,000 Additional Data for predicting which categories have the most oppotunity to get the research prize**

##### 1. extract_data_lasker(url, target_data_count=20, output_csv='extracted_data.csv') 
This part will scrap data from https://laskerfoundation.org/all-awards-winners/ and get all winner detail in each link ~ 337 link
##### 2. extract_data(url, target_data_count=20, output_csv='extracted_data_2.csv') 
This part will scrap data from https://www.nobelprize.org/prizes/lists/all-nobel-prizes/ and get all winner detail in each link ~ 800 link

In [9]:
import os
import json
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
import csv
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
import re
import time

In [ ]:
def extract_data_lasker(url, target_data_count=20, output_csv='extracted_data.csv'):
    print(f"Opening page: {url}")  # Print progress as soon as the page starts loading
    
    chrome_options = Options()
    chrome_options.add_argument("--disable-gpu")  # Disable GPU acceleration (useful for some environments)
    webdriver_path = r"C:\users\asus\Downloads\chromedriver-win64\chromedriver-win64\chromedriver.exe"

    # Create a Service object
    service = Service(webdriver_path)
    
    # Initialize the WebDriver using the Service object
    driver = webdriver.Chrome(service=service, options=chrome_options)

    try:
        driver.get(url)
        data = []
        collected_count = 0  # Counter for the number of collected items
        soup = BeautifulSoup(driver.page_source, "lxml")
        articles = soup.find_all('article')
        while(len(articles) < 500):
            soup = BeautifulSoup(driver.page_source, "lxml")
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(3)
            articles = soup.find_all('article')
            print(len(articles))
            if(len(articles) == 336):break
        for article in articles:
            title_author = article.find('div', {'class': 'fusion-post-content post-content'})
            a_title = title_author.find('a')
            author_name = title_author.find('p')
            title_link = a_title['href'] if a_title else None 


            year_prize_a = article.find('div', {'class' : "fusion-meta-info"})
            year_prize = year_prize_a.find_all('a')
            year = year_prize[1]
            prize = year_prize[2]



            if a_title and author_name:
                data.append([a_title.text.strip(), author_name.text.strip(), year.text.strip(), prize.text.strip(), title_link])  # Append both title and author to the list


    except Exception as e:
        print(f"Error scraping {url}: {e}")  # Log errors during scraping
        data = []
    finally:
        driver.quit()
    
    print(f"Finished scraping: {url}")  # Notify when scraping for this URL is complete
    print(f"Collected {collected_count} items.")
    
#     # Save data to CSV
    save_to_csv(data, output_csv)
    
    return data

def save_to_csv(data, filename):
    """Saves the collected data to a CSV file."""
    try:
        with open(filename, mode='w', newline='', encoding='utf-8') as file:
            writer = csv.writer(file)
            writer.writerow(['Title', 'Author', 'Year', 'Type of Prize', 'url'])  # Write the header row with both Title and Author
            writer.writerows(data)  # Write the collected data
        print(f"Data saved to {filename}")
    except Exception as e:
        print(f"Error saving data to CSV: {e}")

# Example usage
extracted_data = extract_data_lasker("https://laskerfoundation.org/all-awards-winners/")
print(f"Total data collected: {len(extracted_data)}")


Opening page: https://laskerfoundation.org/all-awards-winners/
16
32
48


In [4]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import json

# Set up Selenium WebDriver
chrome_options = Options()
# chrome_options.add_argument("--headless")  # Run in headless mode
chrome_options.add_argument("--disable-gpu")
webdriver_path = r"C:\users\asus\Downloads\chromedriver-win64\chromedriver-win64\chromedriver.exe"

 # Create a Service object
service = Service(webdriver_path)

# Initialize the WebDriver using the Service object
driver = webdriver.Chrome(service=service, options=chrome_options)

# URL of the Scopus record
url = "https://www.scopus.com/record/display.uri?eid=2-s2.0-85179929360&origin=inward"

try:
    # Open the webpage
    driver.get(url)
    
    # Wait for the content to load (adjust time as necessary)
    driver.implicitly_wait(10)
    
    # Extract the title
    title = driver.title.strip()

    # Extract the abstract section
    try:
        abstract_section = driver.find_element(By.ID, "abstractSection")
        abstract = abstract_section.find_element(By.TAG_NAME, "p").text.strip()
    except Exception:
        abstract = "Abstract not found"

    # Extract the author keywords (adjust the selector if necessary)
    try:
        keyword_elements = driver.find_elements(By.CSS_SELECTOR, "#authorKeywords .badges")
        keywords = [keyword.text for keyword in keyword_elements]
    except Exception:
        keywords = []

    # Prepare data for saving
    data = {
        "title": title,
        "abstract": abstract,
        "author_keywords": keywords
    }
    
    # Save data to a JSON file
    with open("scopus_results.json", "w", encoding="utf-8") as json_file:
        json.dump(data, json_file, ensure_ascii=False, indent=4)
    print("Data saved to scopus_results.json")

    # Save the full HTML content to a file
    full_html = driver.page_source
    with open("scopus_full_html.html", "w", encoding="utf-8") as html_file:
        html_file.write(full_html)
    print("Full HTML content saved to scopus_full_html.html")

finally:
    # Close the WebDriver
    driver.quit()

Data saved to scopus_results.json


KeyboardInterrupt: 

In [ ]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

def extract_data(url, output_csv='extracted_data_2.csv'):
    print(f"Opening page: {url}")  # Print progress as soon as the page starts loading
    
    chrome_options = Options()
    chrome_options.add_argument("--disable-gpu")  # Disable GPU acceleration (useful for some environments)
    webdriver_path = r"C:\users\asus\Downloads\chromedriver-win64\chromedriver-win64\chromedriver.exe"

    # Create a Service object
    service = Service(webdriver_path)
    
    # Initialize the WebDriver using the Service object
    driver = webdriver.Chrome(service=service, options=chrome_options)

    try:
        driver.get(url)
        soup = BeautifulSoup(driver.page_source, "html.parser")  # Python HTML parser
        data = []

        articles = soup.find_all('div', {'class': 'card-prize'})
        cookie_found = False

        while(len(articles) < 40): 
            
            if(not cookie_found):
                time.sleep(2)
                cookie_banner = WebDriverWait(driver, 10).until(
                    EC.visibility_of_element_located((By.ID, "onetrust-accept-btn-handler"))
                )
                if(cookie_banner) : 
                    cookie_banner.click()  # Click the "Accept" button or close the banner
                    print("Clicked cookie banner accept button.")
                    cookie_found = True

            # Wait for and click the "Load More" button
            try:
                load_more_button = WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable((By.CLASS_NAME, "dynamic-list-load-more"))
                )
                load_more_button.click()  # Click the 'Load More' button
                print("Clicked 'Load More' button successfully.")
            except Exception as e:
                print("Load More button not found or error:", e)
                break  # If no "Load More" button is found, exit the loop

            # Wait for the page to load more content
            time.sleep(3)  # Wait for 3 seconds for the page to load more data

            # Re-parse the page again after clicking 'Load More'
            soup = BeautifulSoup(driver.page_source, "lxml")

            # Extract articles
            articles = soup.find_all('div', {'class': 'card-prize'})
            print(f"Found {len(articles)} new articles.")
            



            articles = soup.find_all('div', {'class': 'card-prize'})
            print(f"Found {len(articles)} articles so far.")
            for article in articles:
                try:
                    author_section = article.find('div', {'class': 'card-prize--laureates--links'})
                    authors = [author.text.strip() for author in author_section.find_all('a')] if author_section else []

                    title = article.find('blockquote', {'class': 'card-prize--laureates--motivation --last'})
                    title_text = title.text.strip() if title else ""
                    clean_title = title_text.replace("“", "").replace("”", "").replace('"', '').lstrip("for ").strip()

                    element = article.find('h3')
                    element_a = element.find('a') if element else None
                    link = element_a['href'] if element_a else None
                    full_text = element_a.text if element_a else ""

                    parts = full_text.split(" ")
                    if len(parts) >= 3:
                        _, subject, year = parts[-3:]
                    else:
                        subject, year = "Unknown", "Unknown"

                    if clean_title and authors:
                        data.append([clean_title, authors, year, subject + ' Nobel prize', link])

                except Exception as e:
                    print(f"Error processing article: {e}")


    except Exception as e:
        print(f"Error scraping {url}: {e}")  # Log errors during scraping
        data = []
    finally:
        driver.quit()
    
    print(f"Finished scraping: {url}")  # Notify when scraping for this URL is complete
    # Save data to CSV
    save_to_csv(data, output_csv)
    
    return data

def save_to_csv(data, filename):
    """Saves the collected data to a CSV file."""
    try:
        with open(filename, mode='w', newline='', encoding='utf-8') as file:
            writer = csv.writer(file)
            writer.writerow(['Title', 'Author', 'Year', 'Type of Prize', 'url'])  # Write the header row with both Title and Author
            writer.writerows(data)  # Write the collected data
        print(f"Data saved to {filename}")
    except Exception as e:
        print(f"Error saving data to CSV: {e}")
extracted_data = extract_data("https://www.nobelprize.org/prizes/lists/all-nobel-prizes/")
print(f"Total data collected: {len(extracted_data)}")

Opening page: https://www.nobelprize.org/prizes/lists/all-nobel-prizes/
Clicked cookie banner accept button.
Clicked 'Load More' button successfully.
Found 90 new articles.
Found 90 articles so far.
Finished scraping: https://www.nobelprize.org/prizes/lists/all-nobel-prizes/
Data saved to extracted_data_2.csv
Total data collected: 90


In [ ]:
# # import required libraries
# from kafka import KafkaProducer
# import time
# from datetime import datetime

In [ ]:
# # Connect to kafka broker running in your local host (docker). Change this to your kafka broker if needed
# kafka_broker = 'localhost:29092'
# producer = KafkaProducer(
#     bootstrap_servers=[kafka_broker],
#     linger_ms=5000,  # Increased linger time
#     max_block_ms=60000,  # Increase the max block time (default: 60000 ms = 1 minute)
# )